## Correos electronicos SPAM: Un enfoque con Procesamiento de Lenguaje Natural


### Uriel Antonio Alvarez Chavez 16111783

Los correos electronicos no deseados en su bandeja de entrada son molestos ya que perturban la rutina del usuario.Es por eso que las cuentas de correo electronico ya tiene un filtro de spam. Dadoo que es una de las aplicaciones PLN mas utilizadas vamos a ver como se derarrolla un filtro spam simple para correos electronicos.

In [1]:
from functools import reduce

import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import re

In [2]:
#Inserta los datos
full_corpus = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None, names =['label','msg_body'] )

#Separando los mensajes en 'ham'y 'spam'
ham_text=[]
spam_text=[]


### 1. Inspeccio y separacion de mensajes en las categorias "ham" y "spam"

Inicialmente el conjunto de datos debe inspeccionarce para ocuparlo y abordarlo para lograr la tarea. El formato de los datos ddos, la cantidad de datos proporcionados, la naturaleza de los datos se incluyen en esta inspeccion para identificar la mejor aproximacion posible para la tarea.

el corpus de menses dado ha marcado cada mensjae como ham o spam. Ademas hat 5568 mensajes en un Data Frame escrito en ingles que no son objetos nulos. Por lo tanto, el archivo  se puede leer usando DataFrame en python para clasificar esos mensajes de acuerdo con el indicador dado.

In [3]:
def separate_msgs():
    for index,column in full_corpus.iterrows():
        label=column[0]
        message_text=column[1]
        if label =='ham':
            ham_text.append(message_text)
        elif label == 'spam':
                spam_text.append(message_text)
                
separate_msgs()

### 2. Procesamiento de texto

El procesamiento es la tarea de realizar los pasos de preparacion den el corpus de texto sin formato para completar de manera eficiente una extraccion de texto o procesamiento de lenguaje natural o cualquier otra tarea que incluya texto sin formato. El procesamiento de texto consta de varios ppasos, aunque algunos de ellos pueden no aplicarce a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el preprocesamiento de texto inlcuye los siguientes pasos de acuerdo con el conjunto de datos:

### Eliminacion de signos de puntuacion

In [4]:
# Eliminacion de los signos de puntuacion de los mensajes de correo electronico
def remove_msg_punctuations(email_msg):
    puntuation_remove_msg="".join([word for word in email_msg if word not in string.punctuation])
    return puntuation_removed_msg

### Convertir a minusculas:


Convertir a minusculas: La conversion de todos los caracteres del texto en un contexto comun, como los soportes en minusculas impide identificar dos palabaras de manera diferente donde una esta en minuscula y la otra no. Por ejemplo, "Primero" y "primero" deben identificarse como iguales, por lo tanto poner en minuscula todos los caracteres facilita la tarea.Ademas, las palabras de detencion tambien estan en minusculas, por lo que esto tambien haria posibles eliminar palabras de detencion mas adelante.

### Tokenizing

Tokenizing: La tokenizacion es la tarea de dividir el texto en partes significativas, es decir, tokens que incluyen oraciones y palabras. Un token se puede considerar como una instancia de una secuencia de caracteres en un texto particular que se agrupan para proporcionar una unidad semantica util para su posterior procesamiento. En esta tarea, de tokenizacion de palabras se raliza combinado espacios en blanco entre palabras como delimitador.Esto se logra en Python usando expresiones regulares para dividir una cadena en subcadenas con la funcion split(). que es un tokenizador basico.

In [5]:
#Convierte el texto en minusculas y tokenizing de palabras
def tokenize_into_words(text):
    tokens=re.split('\W+',text)
    return tokens

### Palabras


La derivacion es el proceso de eliminar afijos(sufijos, prefijos, infijos, circunfijos) de una palabra para obtener su raiz de palabra. Aunque la lematizacion esta relacionada con la derivacion, difiere ya que la lematizacion puede capturar formas canonicas basadas en el lema de una palabra. La lematizacion ocupa un vocabulario y un analisis morfologico de las palabras que lo hacen mas rapido y preciso que la derivacion. WordNetLemmatizer ha logrado la lematizacion en lenguaje Python.

In [6]:
#Lemmatizing
Word_lemmatizer = WordNetLemmatizer()
def lemmatization(tokenized_words):
    lemmatized_text = [word_lemmatizer.lemmatize(word)for word in tokenized_words]
    return ' '.join(lemmatized_text)

def preprocessing_msgs(corpus):
    categorized_text = pd.DataFrame(corpus)
    categorized_text['non_punc_message_body'] = categorized_text[0].apply(lambda msg: remove_msg_punctuations(msg))
    categorized_text['tokenized_msg_body'] = categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_words'] = categorized_text['tokenized_msg_body'].apply(lambda word_list: lemmatization(word_list))
    return categorized_text['lemmatized_msg_words']